## Prerequisites

This is a simple inference example using Imagenet MobilenetV1 model. To run this examples, you must install the required packages and setup Python environment by following the guides:
* [FuriosaAI Driver, Firmware, Runtime Installation Guide](https://furiosa-ai.github.io/docs/latest/ko/software/installation.html)
* [Setting up a Python Environment](https://furiosa-ai.github.io/docs/latest/ko/software/python-sdk.html#python)

Then, please install the following python packages:
```sh
pip install furiosa-sdk matplotlib pillow
```
Otherwise, you can run the following command to install all dependent packages for notebook examples:
```sh
pip install -r examples/notebooks/requirements.txt
```

Then, you are now ready to run the following examples.

## Checking environment
First, let's check your NPU device is ready.

In [1]:
from PIL import ImageDraw
from PIL import Image

# Import Furiosa SDK
from furiosa.runtime import session

# Import helper for image classlification
from helper import load_labels, image_path, show_image
from helper.mobilenet_ssd import *

import time

libfuriosa_hal.so --- v0.11.0, built @ 43c901f


# Image Classification

In [2]:
# Load and compile an imagenet model for NPU

model_path = "models/imagenet_224x224_mobilenet_v1_uint8_quantization-aware-trained_dm_1.0_without_softmax.tflite"
start_time = time.time()
classification = session.create(model_path)
print('Compilation took {:.2f} secs'.format(time.time() - start_time))

Saving the compilation log into /home/hyunsik/.local/state/furiosa/logs/compile-20230413184339-37zucx.log
Using furiosa-compiler 0.9.0 (rev: 062c7dd1f built at 2023-04-12T20:55:14Z)
[1/6] 🔍   Compiling from tflite to dfg
Done in 0.08141963s
[2/6] 🔍   Compiling from dfg to ldfg


2023-04-13T23:43:39.084323Z  INFO nux::npu: Npu (npu0pe0-1) is being initialized
2023-04-13T23:43:39.086939Z  INFO nux: NuxInner create with pes: [PeId(0)]


Done in 33.84961s
[3/6] 🔍   Compiling from ldfg to cdfg
Done in 0.000877171s
[4/6] 🔍   Compiling from cdfg to gir
Done in 0.007671185s
[5/6] 🔍   Compiling from gir to lir
Done in 0.001998134s
[6/6] 🔍   Compiling from lir to enf
Done in 0.023950694s
✨  Finished in 33.965992s


Compilation took 34.26 secs


In [3]:
# Print out the summary of the model

classification.print_summary()

Inputs:
{0: TensorDesc(shape=(1, 224, 224, 3), dtype=UINT8, format=NHWC, size=150528, len=150528)}
Outputs:
{0: TensorDesc(shape=(1, 1, 1, 1001), dtype=UINT8, format=NHWC, size=1001, len=1001)}


In [4]:
# Get the shape of the model input and resize the input image 
# according to the input tensor shape of the model.

_, height, width, channel = classification.input(0).shape
image = Image.open(image_path('car.jpg')).convert('RGB').resize((width, height))
show_image(image)

In [5]:
# Fill a numpy ndarray with the resized image

data = np.zeros((width, height, channel), np.uint8)
data[:width, :height, :channel] = image
data = np.reshape(data, (1, width, height, channel))

In [6]:
# Run an inference with the numpy ndarray

start_time = time.time()
outputs = classification.run(data)
print('Elapsed {:.2f} secs'.format(time.time() - start_time))

Elapsed 0.00 secs


In [7]:
# Check out the output tensor shape

outputs

{0: Tensor(shape=(1, 1, 1, 1001), dtype=UINT8)}

In [8]:
# Post processing: transform the tensor outputs to human readable data

classified = np.squeeze(outputs[0].numpy())
imagenet_labels = load_labels('ImageNetLabels.txt')
Class = collections.namedtuple('Class', ['id', 'score'])
objects = []
for idx, n in np.ndenumerate(classified):
    objects.append(Class(idx[0], n))
    
objects.sort(key=lambda x: x[1], reverse=True)
print("[Top 5 scores:]")
for object in objects[:5]:
    print("{}: {}".format(imagenet_labels[object.id], object.score))

[Top 5 scores:]
sports car: 155
pickup: 152
car wheel: 148
convertible: 148
racer: 143


In [9]:
classification.close()

2023-04-13T23:44:13.553179Z  INFO nux::npu: NPU (npu0pe0-1) has been destroyed
2023-04-13T23:44:13.553491Z  INFO nux::capi: session has been destroyed
